In [18]:
#import libraries

import os
import pandas as pd
import hvplot
import matplotlib
import requests
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import datetime
import json

In [19]:
load_dotenv()

True

In [20]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")



In [21]:
# Instantiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    #base_url,
    api_version = "v2"
)

In [22]:
# Get user input for the stock symbol
# stock_symbol = input("Enter the stock symbol you want to analyze (e.g., AAPL): ")
stock_symbol = "META"

In [23]:
# Establish what time frame increment we will be looking at
time_frame = "1Day"

In [24]:
# Get user input for the start date in datetime format
# start_date_str = input("Enter the start date (YYYY-MM-DD): ")
# start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
start_date = pd.Timestamp("2020-10-10", tz="America/New_York").isoformat()

In [25]:
# Get user input for the end date in datetime format
# end_date_str = input("Enter the end date (YYYY-MM-DD): ")
# end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
end_date = pd.Timestamp("2020-12-12", tz="America/New_York").isoformat()

In [26]:
# trading_days = end_date - start_date


# print(f"The timeframe between {start_date_str} and {end_date_str} is {trading_days.days} days.")

In [27]:
# Calls the stock data from the period of time desired in the desired time increment
stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

In [28]:
# Displays the information pulled
stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44 entries, 2020-10-12 04:00:00+00:00 to 2020-12-11 05:00:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        44 non-null     float64
 1   high         44 non-null     float64
 2   low          44 non-null     float64
 3   trade_count  44 non-null     int64  
 4   open         44 non-null     float64
 5   volume       44 non-null     int64  
 6   vwap         44 non-null     float64
dtypes: float64(5), int64(2)
memory usage: 2.8 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,292730,270.200,32999660,275.227035
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,181923,277.580,19505211,276.201124
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,151136,277.620,16590158,273.981850
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,182305,267.600,16330684,266.033730
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,157636,267.375,18430693,267.458035


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2020-12-07 05:00:00+00:00,285.58,288.4900,278.20,153281,279.190,14005842,285.751969
2020-12-08 05:00:00+00:00,283.40,286.4272,281.55,136454,286.010,11974657,283.705184
2020-12-09 05:00:00+00:00,277.92,287.6300,271.75,312698,283.660,26765687,278.642389
2020-12-10 05:00:00+00:00,277.12,278.7300,271.86,209825,275.535,21534833,276.018129
2020-12-11 05:00:00+00:00,273.55,276.4800,270.25,158853,274.530,15609459,273.330511


In [29]:
# Add new data columns to DF for body size and candle size
# body size is absolute value of open - close
# candle size is the absolute value of low - high
# clean up datafram by removing vwap, volume, trade count columns


In [30]:
# Drop columns which aren't needed(trade_count, volume, vwap) and display the results
stock_data = stock_data.drop(columns=['trade_count','volume','vwap'])
display(stock_data.head())

,close,high,low,open
timestamp,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,270.200
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,277.580
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,277.620
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,267.600
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,267.375


In [34]:
stock_data.head(2)

,close,high,low,open
timestamp,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,270.20
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,277.58


In [35]:
# Add new data columns to DF for body size and candle size
# body size is absolute value of open - close
# candle size is the absolute value of low - high
# clean up datafram by removing vwap, volume, trade count columns

stock_data['body_size'] = stock_data['close'] - stock_data['open']
stock_data['candle_size'] = stock_data['high'] - stock_data['low']

In [36]:
stock_data.head()

,close,high,low,open,body_size,candle_size
timestamp,,,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,270.200,5.550,12.31
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,277.580,-1.440,5.71
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,277.620,-5.800,7.25
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,267.600,-0.880,5.37
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,267.375,-1.445,6.07
